## 多次调用完成中译英，英文文答，英译中

In [1]:
import openai, os
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

openai.api_key = os.environ.get("OPENAI_API_KEY")

llm = OpenAI(model_name="text-davinci-003", max_tokens=2048, temperature=0.5)

en_to_zh_prompt = PromptTemplate(
    template="请把下面这句话翻译成英文： \n\n {question}?", input_variables=["question"]
)

question_prompt = PromptTemplate(
    template = "{english_question}", input_variables=["english_question"]
)

zh_to_cn_prompt = PromptTemplate(
    input_variables=["english_answer"],
    template="请把下面这一段翻译成中文： \n\n{english_answer}?",
)


question_translate_chain = LLMChain(llm=llm, prompt=en_to_zh_prompt, output_key="english_question")
english = question_translate_chain.run(question="请你作为一个机器学习的专家，介绍一下CNN的原理。")
print(english)

qa_chain = LLMChain(llm=llm, prompt=question_prompt, output_key="english_answer")
english_answer = qa_chain.run(english_question=english)
print(english_answer)

answer_translate_chain = LLMChain(llm=llm, prompt=zh_to_cn_prompt)
answer = answer_translate_chain.run(english_answer=english_answer)
print(answer)




Please as an expert in machine learning, explain the principle of CNN.


Convolutional Neural Networks (CNNs) are a type of artificial neural network used for image recognition, image classification, and object detection. They are used to analyze visual imagery and are considered to be one of the most powerful techniques for deep learning. 

A CNN works by taking an input image, applying a series of convolutional filters, and then pooling the results to create a feature map. This feature map is then used to identify patterns in the image. The convolutional filters are used to detect edges, curves, and other features in the image. The pooling layers are used to reduce the size of the feature map and to increase the accuracy of the model. The output of the CNN is then used to classify the image into one of the predefined classes.


卷积神经网络（CNN）是一种用于图像识别、图像分类和物体检测的人工神经网络。它们用于分析视觉图像，被认为是深度学习最强大的技术之一。

CNN通过输入图像、应用一系列卷积滤波器，然后池化结果来创建特征图，来完成工作。该特征图然后用于识别图像中的模式。卷积滤波器用于检测图像中的边缘、曲线和其他特征。池化层用于减小

## 使用Langchain简化实现

In [2]:
from langchain.chains import SimpleSequentialChain

chinese_qa_chain = SimpleSequentialChain(
    chains=[question_translate_chain, qa_chain, answer_translate_chain], input_key="question",
    verbose=True)
answer = chinese_qa_chain.run(question="请你作为一个机器学习的专家，介绍一下CNN的原理。")
print(answer)



> Entering new SimpleSequentialChain chain...


Please, as an expert in machine learning, explain the principle of CNN.


Convolutional Neural Networks (CNNs) are a type of deep learning neural network used for image recognition and classification. They are based on the idea of applying a filter to an image to extract features from it. This filter is applied to the image in a sliding window fashion and the output of the filter is used as an input to the next layer of the network. This process is repeated until a desired output is achieved. CNNs are used for tasks such as object detection, image segmentation, and image classification. The main advantage of CNNs is that they are able to learn features from the data without requiring any prior knowledge or manual feature engineering.


卷积神经网络（CNN）是一种深度学习神经网络，用于图像识别和分类。它们基于将滤镜应用于图像以从中提取特征的想法。这个滤镜以滑动窗口的方式应用于图像，滤镜的输出用作网络的下一层的输入。重复这个过程，直到达到预期的输出。CNN用于诸如物体检测、图像分割和图像分类等任务。CNN的主要优势在于，它们能够从数据中学习特征，而无需任何先验知识或手动特征工程。

> Finished chain.


卷积神经网络（C

## 多步骤引入不同输入参数

In [3]:
from langchain.chains import SequentialChain

q1_prompt = PromptTemplate(
    input_variables=["year1"],
    template="{year1}年的欧冠联赛的冠军是哪支球队，只说球队名称。"
)
q2_prompt = PromptTemplate(
    input_variables=["year2"],
    template="{year2}年的欧冠联赛的冠军是哪支球队，只说球队名称。"
)
q3_prompt = PromptTemplate(
    input_variables=["team1", "team2"],
    template="{team1}和{team2}哪只球队获得欧冠的次数多一些？"
)
chain1 = LLMChain(llm=llm, prompt=q1_prompt, output_key="team1")
chain2 = LLMChain(llm=llm, prompt=q2_prompt, output_key="team2")
chain3 = LLMChain(llm=llm, prompt=q3_prompt)

sequential_chain = SequentialChain(chains=[chain1, chain2, chain3], input_variables=["year1", "year2"], verbose=True)
answer = sequential_chain.run(year1=2000, year2=2010)
print(answer)



> Entering new SequentialChain chain...

> Finished chain.


拜仁慕尼黑更胜一筹，获得欧冠冠军次数为5次，而其他球队的次数均不超过4次。


## 通过Langchain来完成自动撰写单元测试

In [4]:
from langchain.chains import SequentialChain


def write_unit_test(function_to_test, unit_test_package = "pytest"):
    # 解释源代码的步骤
    explain_code = """"# How to write great unit tests with {unit_test_package}

    In this advanced tutorial for experts, we'll use Python 3.10 and `{unit_test_package}` to write a suite of unit tests to verify the behavior of the following function.
    ```python
    {function_to_test}
    ```

    Before writing any unit tests, let's review what each element of the function is doing exactly and what the author's intentions may have been.
    - First,"""

    explain_code_template = PromptTemplate(
        input_variables=["unit_test_package", "function_to_test"],
        template=explain_code
    )
    explain_code_llm = OpenAI(model_name="text-davinci-002", temperature=0.4, max_tokens=1000, 
            top_p=1, stop=["\n\n", "\n\t\n", "\n    \n"])
    explain_code_step = LLMChain(llm=explain_code_llm, prompt=explain_code_template, output_key="code_explaination")


    # 创建测试计划示例的步骤
    test_plan = """
        
    A good unit test suite should aim to:
    - Test the function's behavior for a wide range of possible inputs
    - Test edge cases that the author may not have foreseen
    - Take advantage of the features of `{unit_test_package}` to make the tests easy to write and maintain
    - Be easy to read and understand, with clean code and descriptive names
    - Be deterministic, so that the tests always pass or fail in the same way

    `{unit_test_package}` has many convenient features that make it easy to write and maintain unit tests. We'll use them to write unit tests for the function above.

    For this particular function, we'll want our unit tests to handle the following diverse scenarios (and under each scenario, we include a few examples as sub-bullets):
    -"""
    test_plan_template = PromptTemplate(
        input_variables=["unit_test_package", "function_to_test", "code_explaination"],
        template= explain_code + "{code_explaination}" + test_plan
    )
    test_plan_llm = OpenAI(model_name="text-davinci-002", temperature=0.4, max_tokens=1000, 
            top_p=1, stop=["\n\n", "\n\t\n", "\n    \n"])
    test_plan_step = LLMChain(llm=test_plan_llm, prompt=test_plan_template, output_key="test_plan")

    # 撰写测试代码的步骤
    starter_comment = "Below, each test case is represented by a tuple passed to the @pytest.mark.parametrize decorator"
    prompt_to_generate_the_unit_test = """

Before going into the individual tests, let's first look at the complete suite of unit tests as a cohesive whole. We've added helpful comments to explain what each line does.
```python
import {unit_test_package}  # used for our unit tests

{function_to_test}

#{starter_comment}"""

    unit_test_template = PromptTemplate(
        input_variables=["unit_test_package", "function_to_test", "code_explaination", "test_plan", "starter_comment"],
        template= explain_code + "{code_explaination}" + test_plan + "{test_plan}" + prompt_to_generate_the_unit_test
    )
    unit_test_llm = OpenAI(model_name="text-davinci-002", temperature=0.4, max_tokens=1000, stop="```")
    unit_test_step = LLMChain(llm=unit_test_llm, prompt=unit_test_template, output_key="unit_test")

    sequential_chain = SequentialChain(chains=[explain_code_step, test_plan_step, unit_test_step], 
                                    input_variables=["unit_test_package", "function_to_test", "starter_comment"], verbose=True)
    answer = sequential_chain.run(unit_test_package=unit_test_package, function_to_test=function_to_test, starter_comment=starter_comment)
    return f"""#{starter_comment}""" + answer


code = """
def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    if hours > 0:
        return f"{hours}h{minutes}min{seconds}s"
    elif minutes > 0:
        return f"{minutes}min{seconds}s"
    else:
        return f"{seconds}s"
"""

import ast

def write_unit_test_automatically(code, retry=3):
    unit_test_code = write_unit_test(code)
    all_code = code + unit_test_code
    tried = 0
    while tried < retry:
        try:
            ast.parse(all_code)
            return all_code
        except SyntaxError as e:
            print(f"Syntax error in generated code: {e}")
            all_code = code + write_unit_test(code)
            tried += 1

print(write_unit_test_automatically(code))

WARNING! stop is not default parameter.
                    stop was transfered to model_kwargs.
                    Please confirm that stop is what you intended.
WARNING! stop is not default parameter.
                    stop was transfered to model_kwargs.
                    Please confirm that stop is what you intended.
WARNING! stop is not default parameter.
                    stop was transfered to model_kwargs.
                    Please confirm that stop is what you intended.




> Entering new SequentialChain chain...

> Finished chain.

def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    if hours > 0:
        return f"{hours}h{minutes}min{seconds}s"
    elif minutes > 0:
        return f"{minutes}min{seconds}s"
    else:
        return f"{seconds}s"
#Below, each test case is represented by a tuple passed to the @pytest.mark.parametrize decorator.
#The first element of the tuple is the name of the test case, and the second element is a list of tuples.
#Each tuple in the list contains the input to the function and the expected output.
@pytest.mark.parametrize("test_case, (input, expected)", [
    # Test positive integers
    ("1 second", [(1, "1s")]),
    ("60 seconds", [(60, "1min")]),
    ("3600 seconds", [(3600, "1h")]),
    ("86400 seconds", [(86400, "1d")]),
    # Test negative integers
    ("-1 second", [(-1, "-1s")]),
    ("-60 seconds", [(-60, "-1min")]),
    ("-3600 seconds", [(-3600, "-1h